In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# from __future__ import absolute_import
import sys
sys.path.append("../")

import os
from copy import copy

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
import seaborn as sns

from random import choice

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor 
from sklearn.grid_search import RandomizedSearchCV ,GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.feature_selection import SelectKBest

from xgboost import XGBRegressor

from scipy.stats import randint as sp_randint

from utils.evaluation_utils import rmsle,rmsle_on_logs,log_pandas,inv_log_pandas,rmsle_score_on_logs
from utils.generic_utils import pickle_out,pickle_in
import utils.preprocessing_utils as prep

In [24]:
data_folder = os.path.join("../","datasets","initial_data_split")

train = pd.read_csv(os.path.join(data_folder,"train.csv"))
valid = pd.read_csv(os.path.join(data_folder,"valid.csv"))
test = pd.read_csv(os.path.join(data_folder,"test.csv"))

X_train = train.drop(["registered","casual"], axis=1)
Y_train = train[["count"]]
Y_train_count = train[["count"]].apply(log_pandas)
Y_train_casual = train[["casual"]].apply(log_pandas)
Y_train_registered = train[["registered"]].apply(log_pandas)

X_valid = valid.drop(["registered","casual"], axis=1)
Y_valid = valid[["count"]]
Y_valid_count = valid[["count"]].apply(log_pandas)
Y_valid_casual = valid[["casual"]].apply(log_pandas)
Y_valid_registered = valid[["registered"]].apply(log_pandas)

X_test = test.drop(["registered","casual"], axis=1)
Y_test  = test[["count"]]
Y_test_count = test[["count"]].apply(log_pandas)
Y_test_casual = test[["casual"]].apply(log_pandas)
Y_test_registered = test[["registered"]].apply(log_pandas)

full_dataset = pd.concat([train,valid,test])
full_dataset = full_dataset.sort_values("datetime")
X_full_dataset = full_dataset.drop(["registered","casual"], axis=1)
Y_full_dataset = full_dataset[["count"]]


print X_train.shape,Y_train.shape
print X_valid.shape,Y_valid.shape
print X_test.shape,Y_test.shape

(7452, 10) (7452, 1)
(1722, 10) (1722, 1)
(2286, 10) (2286, 1)


In [25]:
vars_to_bin = ['atemp','humidity','windspeed']
bins_per_var = [[0,10,20,30,100],[0,20,40,60,120],[0,30,1000]]
binning_params = zip(vars_to_bin,bins_per_var)

binning_steps = [('binning_%s'%var,prep.BinVariable(colname = var,
                                                    bins = b,
                                                    drop_column = False)) for var,b in binning_params]

binned_varnames = ["%s_binned"%var for var,b in binning_params]

get_variables =['weather','date_year','workingday','season','holiday','time_hour',
                'atemp','humidity','date_weekday','date_month','windspeed']
encode_variables =['weather','date_year','workingday','season','holiday']
encode_variables.extend(binned_varnames)

encoding_steps = [[('encode_label_%s'%var,prep.PandasLabelEncoder(colname=var)),\
        ('encode_one_hot_%s'%var,prep.PandasOneHotEncoder(colname=var,
                                                                  drop_colname=True))] for var in encode_variables]
encoding_steps = [en_st for sublist in encoding_steps for en_st in sublist ]

prep_steps = [('extract_times', prep.ExtractTimes()),
               ('get_variables',prep.ExtractColumns(colnames = get_variables))
             ]+\
               binning_steps+\
                encoding_steps

prep_pipe = Pipeline(prep_steps)
prep_pipe.fit(X_train)

Pipeline(steps=[('extract_times', ExtractTimes()), ('get_variables', ExtractColumns(colnames=['weather', 'date_year', 'workingday', 'season', 'holiday', 'time_hour', 'atemp', 'humidity', 'date_weekday', 'date_month', 'windspeed'])), ('binning_atemp', BinVariable(bins=[0, 10, 20, 30, 100], colname='atemp', dr...code_one_hot_windspeed_binned', PandasOneHotEncoder(colname='windspeed_binned', drop_colname=True))])

### All in one

In [4]:
classification_steps_all_xgb = [('xgboost',XGBRegressor(n_estimators=100,
                                                         learning_rate=0.1))]

classification_pipe_all = Pipeline(classification_steps_all_xgb)

param_dist_all= {"xgboost__max_depth": range(2,10,1),
                 "xgboost__min_child_weight":range(1,8,2),
                 "xgboost__gamma": [0,0.1,0.2],
                 "xgboost__subsample": [i/10.0 for i in range(6,10)],
                 "xgboost__colsample_bytree": [i/10.0 for i in range(6,10)],
                 "xgboost__reg_alpha":[1e-5, 1e-2, 0.1, 1, 100]
                }

random_search_all = RandomizedSearchCV(classification_pipe_all, 
                                   param_distributions = param_dist_all,
                                   scoring = make_scorer(rmsle_score_on_logs),
                                   n_iter=3,
                                   n_jobs=3,
                                   cv=2)

X_tr = prep_pipe.transform(X_train)
X_vd = prep_pipe.transform(X_valid)

random_search_all.fit(X_tr,Y_train.apply(log_pandas).values.ravel())

print "Best Params",random_search_all.best_estimator_
Y_pred_valid_count = pd.DataFrame(random_search_all.predict(X_vd)).apply(inv_log_pandas)
valid_error_count = rmsle(Y_pred_valid_count.values.ravel(),
                           Y_valid_count.apply(inv_log_pandas).values.ravel())
print "Best model error on valid",valid_error_count

Best Params Pipeline(steps=[('xgboost', XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.2, learning_rate=0.1, max_delta_step=0, max_depth=9,
       min_child_weight=5, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=1e-05, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.9))])
Best model error on valid 0.258991017521


In [5]:
for a in random_search_all.grid_scores_:
    print a

mean: -0.56063, std: 0.06138, params: {'xgboost__colsample_bytree': 0.8, 'xgboost__max_depth': 9, 'xgboost__min_child_weight': 5, 'xgboost__reg_alpha': 1e-05, 'xgboost__subsample': 0.9, 'xgboost__gamma': 0.2}
mean: -0.72017, std: 0.01943, params: {'xgboost__colsample_bytree': 0.9, 'xgboost__max_depth': 2, 'xgboost__min_child_weight': 7, 'xgboost__reg_alpha': 1, 'xgboost__subsample': 0.6, 'xgboost__gamma': 0}
mean: -0.56310, std: 0.05935, params: {'xgboost__colsample_bytree': 0.6, 'xgboost__max_depth': 4, 'xgboost__min_child_weight': 3, 'xgboost__reg_alpha': 1e-05, 'xgboost__subsample': 0.8, 'xgboost__gamma': 0.2}


### Following tutorial 

In [14]:
classification_steps_tree_specific_xgb = [('xgboost',XGBRegressor(n_estimators=200,
                                                                  learning_rate=0.1,
                                                                  gamma = 0,
                                                                  subsample=0.8, 
                                                                  colsample_bytree=0.8,
                                                                  scale_pos_weight=1))]

classification_pipe_tree_specific = Pipeline(classification_steps_tree_specific_xgb)

param_dist_tree_specific = {"xgboost__max_depth": [3,5,10],
                            "xgboost__min_child_weight":range(1,6,2)
                           }

grid_search_tree_specific = GridSearchCV(classification_pipe_tree_specific, 
                                   param_grid = param_dist_tree_specific,
                                   scoring = make_scorer(rmsle_score_on_logs),
                                   n_jobs=3,
                                   cv=10)

X_tr = prep_pipe.transform(X_train)
X_vd = prep_pipe.transform(X_valid)

grid_search_tree_specific.fit(X_tr,Y_train.apply(log_pandas).values.ravel())

print "Best Params",grid_search_tree_specific.best_estimator_
Y_pred_valid_count = pd.DataFrame(grid_search_tree_specific.predict(X_vd)).apply(inv_log_pandas)
valid_error_count = rmsle(Y_pred_valid_count.values.ravel(),
                           Y_valid_count.apply(inv_log_pandas).values.ravel())
print "Best model error on valid",valid_error_count

Best Params Pipeline(steps=[('xgboost', XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8))])
Best model error on valid 0.263076420971


In [21]:
grid_search_tree_specific.grid_scores_

[mean: 0.35144, std: 0.05440, params: {'xgboost__max_depth': 3, 'xgboost__min_child_weight': 1},
 mean: 0.35214, std: 0.05594, params: {'xgboost__max_depth': 3, 'xgboost__min_child_weight': 3},
 mean: 0.35140, std: 0.05404, params: {'xgboost__max_depth': 3, 'xgboost__min_child_weight': 5},
 mean: 0.34499, std: 0.06347, params: {'xgboost__max_depth': 5, 'xgboost__min_child_weight': 1},
 mean: 0.34500, std: 0.06571, params: {'xgboost__max_depth': 5, 'xgboost__min_child_weight': 3},
 mean: 0.35057, std: 0.07124, params: {'xgboost__max_depth': 5, 'xgboost__min_child_weight': 5},
 mean: 0.38001, std: 0.07084, params: {'xgboost__max_depth': 10, 'xgboost__min_child_weight': 1},
 mean: 0.37012, std: 0.06639, params: {'xgboost__max_depth': 10, 'xgboost__min_child_weight': 3},
 mean: 0.37849, std: 0.08033, params: {'xgboost__max_depth': 10, 'xgboost__min_child_weight': 5}]

In [22]:
grid_search_tree_specific.best_estimator_

Pipeline(steps=[('xgboost', XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8))])

In [23]:
classification_steps_gamma_xgb = [('xgboost',XGBRegressor(n_estimators=200,
                                                         learning_rate=0.1,
                                                         max_depth = 5,
                                                         min_child_weight = 3,
                                                         subsample=0.8, 
                                                         colsample_bytree=0.8
                                                         ))]

classification_pipe_gamma = Pipeline(classification_steps_gamma_xgb)

param_grid_gamma = { "xgboost__gamma": [0,0.1,0.2]}

grid_search_gamma = GridSearchCV(classification_pipe_gamma, 
                                   param_grid=param_grid_gamma,
                                   scoring = make_scorer(rmsle_score_on_logs),
                                   n_jobs=3,
                                   cv=10)

X_tr = prep_pipe.transform(X_train)
X_vd = prep_pipe.transform(X_valid)

grid_search_gamma.fit(X_tr,Y_train.apply(log_pandas).values.ravel())

print "Best Params",grid_search_gamma.best_estimator_
Y_pred_valid_count = pd.DataFrame(grid_search_gamma.predict(X_vd)).apply(inv_log_pandas)
valid_error_count = rmsle(Y_pred_valid_count.values.ravel(),
                           Y_valid_count.apply(inv_log_pandas).values.ravel())
print "Best model error on valid",valid_error_count

Best Params Pipeline(steps=[('xgboost', XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=3, missing=None, n_estimators=200, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8))])
Best model error on valid 0.270516764698


In [24]:
grid_search_gamma.grid_scores_

[mean: 0.34500, std: 0.06571, params: {'xgboost__gamma': 0},
 mean: 0.34289, std: 0.06143, params: {'xgboost__gamma': 0.1},
 mean: 0.34358, std: 0.06095, params: {'xgboost__gamma': 0.2}]

In [25]:
classification_steps_subsample_xgb = [('xgboost',XGBRegressor(n_estimators=200,
                                                         learning_rate=0.1,
                                                         gamma = 0.1,
                                                         max_depth = 5,
                                                         min_child_weight = 3,
                                                             ))]

classification_pipe_subsample = Pipeline(classification_steps_subsample_xgb)

param_grid_subsample = { "xgboost__subsample": [i/10.0 for i in range(6,10)],
                    "xgboost__colsample_bytree": [i/10.0 for i in range(6,10)],
                   }

grid_search_subsample = GridSearchCV(classification_pipe_gamma, 
                                   param_grid=param_grid_subsample,
                                   scoring = make_scorer(rmsle_score_on_logs),
                                   n_jobs=3,
                                   cv=10)

X_tr = prep_pipe.transform(X_train)
X_vd = prep_pipe.transform(X_valid)

grid_search_subsample.fit(X_tr,Y_train.apply(log_pandas).values.ravel())

print "Best Params",grid_search_subsample.best_estimator_
Y_pred_valid_count = pd.DataFrame(grid_search_subsample.predict(X_vd)).apply(inv_log_pandas)
valid_error_count = rmsle(Y_pred_valid_count.values.ravel(),
                           Y_valid_count.apply(inv_log_pandas).values.ravel())
print "Best model error on valid",valid_error_count

Best Params Pipeline(steps=[('xgboost', XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=3, missing=None, n_estimators=200, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.6))])
Best model error on valid 0.274256770664


In [26]:
grid_search_subsample.grid_scores_

[mean: 0.34609, std: 0.06228, params: {'xgboost__colsample_bytree': 0.6, 'xgboost__subsample': 0.6},
 mean: 0.33782, std: 0.05530, params: {'xgboost__colsample_bytree': 0.6, 'xgboost__subsample': 0.7},
 mean: 0.33953, std: 0.06022, params: {'xgboost__colsample_bytree': 0.6, 'xgboost__subsample': 0.8},
 mean: 0.33716, std: 0.05810, params: {'xgboost__colsample_bytree': 0.6, 'xgboost__subsample': 0.9},
 mean: 0.34137, std: 0.05605, params: {'xgboost__colsample_bytree': 0.7, 'xgboost__subsample': 0.6},
 mean: 0.33908, std: 0.05566, params: {'xgboost__colsample_bytree': 0.7, 'xgboost__subsample': 0.7},
 mean: 0.34142, std: 0.05885, params: {'xgboost__colsample_bytree': 0.7, 'xgboost__subsample': 0.8},
 mean: 0.34441, std: 0.05879, params: {'xgboost__colsample_bytree': 0.7, 'xgboost__subsample': 0.9},
 mean: 0.34666, std: 0.06055, params: {'xgboost__colsample_bytree': 0.8, 'xgboost__subsample': 0.6},
 mean: 0.34511, std: 0.06230, params: {'xgboost__colsample_bytree': 0.8, 'xgboost__subsampl

In [4]:
classification_steps_regul_xgb = [('xgboost',XGBRegressor(n_estimators=200,
                                                         learning_rate=0.1,
                                                         gamma = 0.1,
                                                         max_depth = 5,
                                                         min_child_weight = 3,
                                                         subsample = 0.9,
                                                         colsample_bytree = 0.6,
                                                             ))]

classification_pipe_regul = Pipeline(classification_steps_regul_xgb)

param_grid_regul = { "xgboost__reg_alpha":[1e-5, 1e-2, 0.1, 1, 100]}

grid_search_regul = GridSearchCV(classification_pipe_regul, 
                                   param_grid=param_grid_regul,
                                   scoring = make_scorer(rmsle_score_on_logs),
                                   n_jobs=3,
                                   cv=10)

X_tr = prep_pipe.transform(X_train)
X_vd = prep_pipe.transform(X_valid)

grid_search_regul.fit(X_tr,Y_train.apply(log_pandas).values.ravel())

print "Best Params",grid_search_regul.best_estimator_
Y_pred_valid_count = pd.DataFrame(grid_search_regul.predict(X_vd)).apply(inv_log_pandas)
valid_error_count = rmsle(Y_pred_valid_count.values.ravel(),
                           Y_valid_count.apply(inv_log_pandas).values.ravel())
print "Best model error on valid",valid_error_count

Best Params Pipeline(steps=[('xgboost', XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=0.1, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=3, missing=None, n_estimators=200, nthread=-1,
       objective='reg:linear', reg_alpha=100, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.9))])
Best model error on valid 0.418528839212


In [5]:
grid_search_regul.grid_scores_

[mean: 0.34230, std: 0.06397, params: {'xgboost__reg_alpha': 1e-05},
 mean: 0.34137, std: 0.06443, params: {'xgboost__reg_alpha': 0.01},
 mean: 0.33850, std: 0.05925, params: {'xgboost__reg_alpha': 0.1},
 mean: 0.33431, std: 0.05660, params: {'xgboost__reg_alpha': 1},
 mean: 0.49809, std: 0.08508, params: {'xgboost__reg_alpha': 100}]

In [26]:
classification_steps_final_xgb = [('xgboost',XGBRegressor(n_estimators=2000,
                                                         learning_rate=0.01,
                                                         gamma = 0.1,
                                                         max_depth = 5,
                                                         min_child_weight = 3,
                                                         subsample = 0.9,
                                                         colsample_bytree = 0.6,
                                                         reg_alpha= 1
                                                             ))]

classification_pipe_final = Pipeline(classification_steps_final_xgb)

In [17]:
final_steps = prep_steps+classification_steps_final_xgb
final_pipe = Pipeline(final_steps)

final_pipe.fit(X_train,Y_train.apply(log_pandas).values.ravel())

Y_pred_valid_count = pd.DataFrame(final_pipe.predict(X_valid)).apply(inv_log_pandas)
valid_error_count = rmsle(Y_pred_valid_count.values.ravel(),
                       Y_valid_count.apply(inv_log_pandas).values.ravel()
                          )
print "Final model valid results:",valid_error_count

Y_pred_test_count = pd.DataFrame(final_pipe.predict(X_test)).apply(inv_log_pandas)
test_error_count = rmsle(Y_pred_test_count.values.ravel(),
                       Y_test_count.apply(inv_log_pandas).values.ravel()
                          )
print "Final model test results:",test_error_count

Final model valid results: 0.267706816155
Final model test results: 0.307521689014


In [28]:
final_steps = prep_steps+classification_steps_final_xgb
final_pipe = Pipeline(final_steps)

final_pipe.fit(X_full_dataset,Y_full_dataset.apply(log_pandas).values.ravel())

Y_pred_test_count = pd.DataFrame(final_pipe.predict(X_full_dataset)).apply(inv_log_pandas)
test_error_count = rmsle(Y_pred_test_count.values.ravel(),
                       Y_full_dataset.values.ravel()
                          )
print "Final model test results:",test_error_count

Final model test results: 0.24061306528


In [31]:
model_filepath = os.path.join("../","models","xgboost_trained_on_full_pipeline.pkl")
pickle_out(model_filepath,final_pipe,compresion_mode=5)

In [32]:
kaggle_folder = os.path.join("../","datasets","kaggle_sets")
kaggle_test = pd.read_csv(os.path.join(kaggle_folder,"test.csv"))
full_pipeline = pickle_in(os.path.join("../","models","xgboost_trained_on_full_pipeline.pkl"),
                          compresion_mode=5)

X_test = kaggle_test
kaggle_datetime = kaggle_test[["datetime"]]
Y_kaggle = pd.DataFrame(full_pipeline.predict(X_test)).apply(inv_log_pandas)
kaggle_datetime["count"] = Y_kaggle
kaggle_datetime.to_csv(os.path.join("../","submissions",
                                    "xgboost_trained_on_full_submission.csv"),index=False)

print kaggle_datetime.head()

              datetime      count
0  2011-01-20 00:00:00  10.912942
1  2011-01-20 01:00:00   4.623943
2  2011-01-20 02:00:00   2.538523
3  2011-01-20 03:00:00   1.740253
4  2011-01-20 04:00:00   1.739652


/home/jakubczakon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
